# Tips for Data Scientists to Get Started with GPU Acceleration

## Introduction
This notebook showcases important functionalities that are important for data scientist and how RAPIDS accelerates workflows using its powerful suite of libraries and frameworks.  

## Data We'll be Using
We'll be exploring and augmenting the Titanic passenger demographic data set from Kaggle to showcase how you can apply these functions to yoru real world data.  The dataset used for this notebook can be downloaded from Kaggle and consists of a 
- [train](https://www.kaggle.com/code/startupsci/titanic-data-science-solutions/input?select=train.csv) dataset
- [test](https://www.kaggle.com/code/startupsci/titanic-data-science-solutions/input?select=test.csv) dataset

You will need to accept the terms of the competition before you can download it.  Once you do, please download both before continuing and put them into the same folder as you're running this notebook.








## Activate GPU-Acceleration

%load_ext cudf.pandas loads the cuDF extension for Pandas, allowing the use of GPU-accelerated DataFrames.

In [1]:
# %load_ext cudf.pandas

## Import Data

Import libraries, read Titanic data, and concatenate data

In [1]:
import pandas as pd
# import cupy as cp
import numpy as np

train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')
concat = pd.concat([train, test], axis = 0)

Scale up the dataset to demonstrate the advantage of GPU acceleration: the original Titanic dataset is too small, so we replicate it to simulate a dataset with 1 million rows

In [2]:
target_rows = 1_000_000
repeats = -(-target_rows // len(train))  # Ceiling division
train_df = pd.concat([train] * repeats, ignore_index=True).head(target_rows)
print(train_df.shape)  # (1000000, 2)

repeats = -(-target_rows // len(test))  # Ceiling division
test_df = pd.concat([test] * repeats, ignore_index=True).head(target_rows)
print(test_df.shape)  # (1000000, 2)

combine = [train_df, test_df]

(1000000, 12)
(1000000, 11)


## Feature Engineering

For the feature engineering steps in this notebook, we borrowed and adapted code from the [Titanic Data Science Solutions](https://www.kaggle.com/code/startupsci/titanic-data-science-solutions) notebook. However, we made the feature engineering GPU-accelerated using RAPIDS libraries to significantly enhance performance. Some of the feature engineering decisions were derived from the Exploratory Data Analysis (EDA) step in the original notebook. Since EDA is beyond the scope of this notebook, we have not included those steps here. If you're interested in exploring the detailed EDA process, we encourage you to visit the original notebook for more insights.

Drop features that do not contribute to the predictions

In [3]:
print("Before", train_df.shape, test_df.shape, combine[0].shape, combine[1].shape)

train_df = train_df.drop(['Ticket', 'Cabin'], axis=1)
test_df = test_df.drop(['Ticket', 'Cabin'], axis=1)
combine = [train_df, test_df]

print("After", train_df.shape, test_df.shape, combine[0].shape, combine[1].shape)

Before (1000000, 12) (1000000, 11) (1000000, 12) (1000000, 11)
After (1000000, 10) (1000000, 9) (1000000, 10) (1000000, 9)


Extract 'Title' from 'Name'

In [4]:
for dataset in combine:
    dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\\.', expand=False)

pd.crosstab(train_df['Title'], train_df['Sex'])

Sex,female,male
Title,,
Capt,0,1122
Col,0,2244
Countess,1122,0
Don,0,1123
Dr,1122,6733
Jonkheer,0,1122
Lady,1122,0
Major,0,2244
Master,0,44898


Replace many titles with a more common name or classify them as 'Rare'

In [5]:
for dataset in combine:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col',\
 	'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
    
train_df[['Title', 'Survived']].groupby(['Title'], as_index=False).mean()

,Title,Survived
0,Master,0.574925
1,Miss,0.702693
2,Mr,0.156669
3,Mrs,0.793644
4,Rare,0.347759


Convert the categorical titles to ordinal

In [6]:
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
for dataset in combine:
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)

train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,S,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C,3
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,S,2
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,S,3
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,S,1


Safely drop 'Name' anad 'PassengerID' feature from training and testing datasets

In [7]:
train_df = train_df.drop(['Name', 'PassengerId'], axis=1)
test_df = test_df.drop(['Name'], axis=1)
combine = [train_df, test_df]
train_df.shape, test_df.shape

((1000000, 9), (1000000, 9))

Convert 'Sex' feature to a new feature 'Gender' when female=1 and male=0

In [8]:
for dataset in combine:
    dataset['Sex'] = dataset['Sex'].map( {'female': 1, 'male': 0} ).astype(int)

train_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,0,3,0,22.0,1,0,7.2500,S,1
1,1,1,1,38.0,1,0,71.2833,C,3
2,1,3,1,26.0,0,0,7.9250,S,2
3,1,1,1,35.0,1,0,53.1000,S,3
4,0,3,0,35.0,0,0,8.0500,S,1


Prepare an empty array to contain guessed Age values based on Pclass x Gender combinations

In [9]:
guess_ages = np.zeros((2,3))
guess_ages

array([[0., 0., 0.],
       [0., 0., 0.]])

We iterate over 'Sex' (0 or 1) and Pclass (1, 2, 3) to calculate guessed values of 'Age' for the six combinations

In [10]:
for dataset in combine:
    for i in range(0, 2):
        for j in range(0, 3):
            guess_df = dataset.loc[(dataset['Sex'] == i) & (dataset['Pclass'] == j+1), 'Age'].dropna()

            # age_mean = guess_df.mean()
            # age_std = guess_df.std()
            # age_guess = rnd.uniform(age_mean - age_std, age_mean + age_std)
            
            age_guess = guess_df.median()

            # Convert random age float to nearest .5 age
            guess_ages[i, j] = int( age_guess/0.5 + 0.5 ) * 0.5
            
            mask = (dataset['Age'].isnull()) & (dataset['Sex'] == i) & (dataset['Pclass'] == j + 1)
            
            dataset.loc[mask, 'Age'] = guess_ages[i, j]
            # .get()

    dataset['Age'] = dataset['Age'].astype(int)

train_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,0,3,0,22,1,0,7.2500,S,1
1,1,1,1,38,1,0,71.2833,C,3
2,1,3,1,26,0,0,7.9250,S,2
3,1,1,1,35,1,0,53.1000,S,3
4,0,3,0,35,0,0,8.0500,S,1


Create Age bands and determine correlations with 'Survived'

In [11]:
train_df['AgeBand'] = pd.cut(train_df['Age'], 5)
train_df[['AgeBand', 'Survived']].groupby(['AgeBand'], as_index=False, observed = True).mean().sort_values(by='AgeBand', ascending=True)

,AgeBand,Survived
0,"(-0.08, 16.0]",0.549973
1,"(16.0, 32.0]",0.337376
2,"(32.0, 48.0]",0.412019
3,"(48.0, 64.0]",0.434755
4,"(64.0, 80.0]",0.090872


Replace 'Age' with ordinals based on these bands

In [12]:
for dataset in combine:    
    dataset.loc[ dataset['Age'] <= 16, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 64, 'Age']
train_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,AgeBand
0,0,3,0,1,1,0,7.2500,S,1,"(16.0, 32.0]"
1,1,1,1,2,1,0,71.2833,C,3,"(32.0, 48.0]"
2,1,3,1,1,0,0,7.9250,S,2,"(16.0, 32.0]"
3,1,1,1,2,1,0,53.1000,S,3,"(32.0, 48.0]"
4,0,3,0,2,0,0,8.0500,S,1,"(32.0, 48.0]"


Remove 'AgeBand' feature 

In [13]:
train_df = train_df.drop(['AgeBand'], axis=1)
combine = [train_df, test_df]
train_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,0,3,0,1,1,0,7.2500,S,1
1,1,1,1,2,1,0,71.2833,C,3
2,1,3,1,1,0,0,7.9250,S,2
3,1,1,1,2,1,0,53.1000,S,3
4,0,3,0,2,0,0,8.0500,S,1


Create a new feature for 'FamilySize' which combines 'Parch' and 'SibSp'. This will allow us to drop 'Parch' and 'SibSp'.

In [14]:
for dataset in combine:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1

train_df[['FamilySize', 'Survived']].groupby(['FamilySize'], as_index=False).mean().sort_values(by='Survived', ascending=False)

,FamilySize,Survived
3,4,0.724127
2,3,0.578411
1,2,0.552758
6,7,0.333457
0,1,0.303542
4,5,0.199929
5,6,0.136349
7,8,0.000000
8,11,0.000000


Create another feature called 'IsAlone'

In [15]:
for dataset in combine:
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1

train_df[['IsAlone', 'Survived']].groupby(['IsAlone'], as_index=False).mean()

,IsAlone,Survived
0,0,0.505619
1,1,0.303542


Drop 'Parch', 'SibSp', and 'FamilySize' features in favor of 'IsAlone'

In [16]:
train_df = train_df.drop(['Parch', 'SibSp', 'FamilySize'], axis=1)
test_df = test_df.drop(['Parch', 'SibSp', 'FamilySize'], axis=1)
combine = [train_df, test_df]

train_df.head()

,Survived,Pclass,Sex,Age,Fare,Embarked,Title,IsAlone
0,0,3,0,1,7.2500,S,1,0
1,1,1,1,2,71.2833,C,3,0
2,1,3,1,1,7.9250,S,2,1
3,1,1,1,2,53.1000,S,3,0
4,0,3,0,2,8.0500,S,1,1


Create an artificial feature combining 'Pclass' and 'Age'

In [17]:
for dataset in combine:
    dataset['Age*Class'] = dataset.Age * dataset.Pclass

train_df.loc[:, ['Age*Class', 'Age', 'Pclass']].head(10)

,Age*Class,Age,Pclass
0,3,1,3
1,2,2,1
2,3,1,3
3,2,2,1
4,6,2,3
5,3,1,3
6,3,3,1
7,0,0,3
8,3,1,3
9,0,0,2


'Embarked' feature takes S, Q, C values based on port of embarkation. Our training dataset has two missing values. We simply fill these with the most common occurance.

In [18]:
freq_port = train_df.Embarked.dropna().mode()[0]
freq_port

'S'

In [19]:
for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].fillna(freq_port)
    
train_df[['Embarked', 'Survived']].groupby(['Embarked'], as_index=False).mean().sort_values(by='Survived', ascending=False)

,Embarked,Survived
0,C,0.553533
1,Q,0.389642
2,S,0.339005


Convert the categorical 'Embarked' feature into numeric

In [20]:
for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)

train_df.head()

,Survived,Pclass,Sex,Age,Fare,Embarked,Title,IsAlone,Age*Class
0,0,3,0,1,7.2500,0,1,0,3
1,1,1,1,2,71.2833,1,3,0,2
2,1,3,1,1,7.9250,0,2,1,3
3,1,1,1,2,53.1000,0,3,0,2
4,0,3,0,2,8.0500,0,1,1,6


Round off the fare to two decimals as it represents currency

In [21]:
test_df['Fare'].fillna(test_df['Fare'].dropna().median(), inplace=True)
test_df.head()

/tmp/ipykernel_1061891/31889810.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  test_df['Fare'].fillna(test_df['Fare'].dropna().median(), inplace=True)


,PassengerId,Pclass,Sex,Age,Fare,Embarked,Title,IsAlone,Age*Class
0,892,3,0,2,7.8292,2,1,1,6
1,893,3,1,2,7.0000,0,3,0,6
2,894,2,0,3,9.6875,2,1,1,6
3,895,3,0,1,8.6625,0,1,1,3
4,896,3,1,1,12.2875,0,3,0,3


Create 'FareBand' feature

In [22]:
num_bins = 4
quantiles = np.percentile(np.array(train_df['Fare']), q=np.linspace(0, 100, num_bins + 1))
bin_indices = np.digitize(np.array(train_df['Fare']), bins=quantiles, right=True) - 1
labels = [f"({np.round(quantiles[i], 3)}, {np.round(quantiles[i+1], 3)}]" for i in range(0, num_bins)]
train_df['FareBand'] = pd.Series([labels[int(i)] if 0 <= i < len(labels) else None for i in bin_indices])
train_df[['FareBand', 'Survived']].groupby(['FareBand'], as_index=False).mean().sort_values(by='FareBand', ascending=True)

,FareBand,Survived
0,"(0.0, 7.896]",0.206753
1,"(14.454, 31.0]",0.454943
2,"(31.0, 512.329]",0.581045
3,"(7.896, 14.454]",0.303565


Convert the 'Fare' feature to ordinal values based on the 'FareBand'

In [23]:
for dataset in combine:
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[ dataset['Fare'] > 31, 'Fare'] = 3
    dataset['Fare'] = dataset['Fare'].astype(int)

train_df = train_df.drop(['FareBand'], axis=1)
combine = [train_df, test_df]
    
train_df.head(10)

,Survived,Pclass,Sex,Age,Fare,Embarked,Title,IsAlone,Age*Class
0,0,3,0,1,0,0,1,0,3
1,1,1,1,2,3,1,3,0,2
2,1,3,1,1,1,0,2,1,3
3,1,1,1,2,3,0,3,0,2
4,0,3,0,2,1,0,1,1,6
5,0,3,0,1,1,2,1,1,3
6,0,1,0,3,3,0,1,1,3
7,0,3,0,0,2,0,4,0,0
8,1,3,1,1,1,0,3,0,3
9,1,2,1,0,2,1,3,0,0


## Machine Learning 

Machine learning workflows often involve computationally intensive operations, particularly when working with large datasets. Traditional CPU-based libraries like scikit-learn are widely used but may struggle to deliver optimal performance at scale. NVIDIA's cuML, a GPU-accelerated machine learning library, addresses this challenge by leveraging the power of GPUs to significantly accelerate common machine learning algorithms while maintaining compatibility with scikit-learn.

In this guide, we explore how to use cuML for key machine learning algorithms, comparing them to their scikit-learn counterparts. The algorithms covered include:

- Logistic Regression: A linear model for binary or multi-class classification, useful for tasks where the target variable is categorical.
- Support Vector Classifier (SVC): A kernel-based method for classification tasks, ideal for handling non-linear boundaries in feature space.
- Linear Support Vector Classifier (LinearSVC): A variant of SVC optimized for linear decision boundaries, often more scalable for large datasets.
- Random Forest Classifier: An ensemble learning method that builds multiple decision trees and combines their outputs for robust and accurate classification.
- K-Nearest Neighbor (KNN) Classifier: A non-parametric algorithm that assigns a class to a data point based on the majority class of its nearest neighbors.
- Gaussian Naive Bayes: A probabilistic classifier that assumes feature independence and fits Gaussian distributions to features for classification.


Identify dependent variables and independent variable

In [24]:
X_train = train_df.drop("Survived", axis=1)
Y_train = train_df["Survived"]
X_test  = test_df.drop("PassengerId", axis=1).copy()
X_train.shape, Y_train.shape, X_test.shape

((1000000, 8), (1000000,), (1000000, 8))

Convert the datatype into 'float32' so that data is better suited for cuML

In [26]:
# X_train = X_train.astype('float32')
# Y_train = Y_train.astype('float32')
# X_test = X_test.astype('float32')

Import cuML libraries for Logistic Regression, Support Vector Classifier, Linear Support Vector Classifier, Random Forest Classifier, K-Nearest Neighbor Classifier, and Gaussian Naive Bayes

In [25]:
# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier


Logistic Regression

In [26]:
%%time
# Logistic Regression

logreg = LogisticRegression(C = 0.1, tol=1e-3)
logreg.fit(X_train, Y_train)
Y_pred = logreg.predict(X_test)
acc_log = np.round(logreg.score(X_train, Y_train) * 100, 2)
acc_log

CPU times: user 17.3 s, sys: 36.9 ms, total: 17.3 s
Wall time: 588 ms


np.float64(80.36)

Support Vector Machines

In [29]:
%%time
# Support Vector Machines

svc = SVC()
svc.fit(X_train, Y_train)
Y_pred = svc.predict(X_test)
acc_svc = np.round(svc.score(X_train, Y_train) * 100, 2)
acc_svc

CPU times: user 5h 57min 33s, sys: 1.84 s, total: 5h 57min 35s
Wall time: 5h 57min 32s


np.float64(83.5)

K-Nearest Neighbors

In [ ]:
%%time
# K-Nearest Neighbors

knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, Y_train)
Y_pred = knn.predict(X_test)
acc_knn = np.round(knn.score(X_train, Y_train) * 100, 2)
acc_knn

Gaussian Naive Bayes

In [31]:
# %%time
# # Gaussian Naive Bayes

# gaussian = GaussianNB()
# gaussian.fit(X_train, Y_train)
# Y_pred = gaussian.predict(cp.array(X_test))
# acc_gaussian = cp.round(gaussian.score(X_train, Y_train) * 100, 2)
# acc_gaussian

Linear SVC

In [32]:
# %%time
# # Linear SVC

# linear_svc = LinearSVC()
# linear_svc.fit(cp.array(X_train), cp.array(Y_train))
# Y_pred = linear_svc.predict(cp.array(X_test))
# acc_linear_svc = cp.round(linear_svc.score(cp.array(X_train), cp.array(Y_train)) * 100, 2)
# acc_linear_svc

Random Forest

In [33]:
# %%time
# # Random Forest

# random_forest = RandomForestClassifier(n_estimators=100)
# random_forest.fit(cp.array(X_train), cp.array(Y_train))
# Y_pred = random_forest.predict(cp.array(X_test))
# random_forest.score(cp.array(X_train), cp.array(Y_train))
# acc_random_forest = cp.round(random_forest.score(cp.array(X_train), cp.array(Y_train)) * 100, 2)
# acc_random_forest

## Ensemble Methods - Stacking Generalization

In [34]:
# from cuml.model_selection import train_test_split, StratifiedKFold

# # Step 2: Define base models (level-0 models)
# base_models = [
#     ("logistic_regression", LogisticRegression(C = 0.1, tol=1e-3)), #80.36
#     ("random_forest", RandomForestClassifier(n_estimators=100)), #81.71
#     ("svc", SVC()),  # Enable probability output for stacking #83.05
# ]

In [35]:
# # Step 3: Generate out-of-fold predictions for stacking
# def generate_meta_features(base_models, X, y, X_meta):
#     kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
#     meta_features = cp.zeros((X_meta.shape[0], len(base_models)))
#     print("Meta features shape:", cp.shape(meta_features))
    
#     for i, (name, model) in enumerate(base_models):
#         meta_predictions = cp.zeros((X.shape[0],))
#         print("Model name: ", name)

#         for train_idx, val_idx in kfold.split(X, y):
#             model.fit(cp.array(X.iloc[train_idx]), cp.array(y.iloc[train_idx]))
#             predictions = model.predict(cp.array(X.iloc[val_idx]))
            
#             # Ensure predictions are 1D
#             meta_predictions[val_idx] = predictions.ravel()
        
#         # Train the model on the full dataset to predict meta features for test data
#         model.fit(cp.array(X), cp.array(y))


#         # Predict meta features for test data
#         predictions = model.predict(cp.array(X_meta))
#         meta_features[:, i] = predictions.ravel()
    
#     return meta_features

# # Generate meta features for training and testing
# meta_train = generate_meta_features(base_models, X_train, Y_train, X_train)
# meta_test = generate_meta_features(base_models, X_train, Y_train, X_test)

In [36]:
# from sklearn.metrics import accuracy_score

# # Step 4: Define the meta-model (level-1 model)
# knn = KNeighborsClassifier(n_neighbors = 3)
# meta_model = knn

# # Train the meta-model
# meta_model.fit(meta_train, Y_train)

# # Step 5: Make predictions with the meta-model
# Y_pred = meta_model.predict(meta_test)

# # Evaluate the performance
# accuracy = cp.round(meta_model.score(meta_train, Y_train)*100, 2)
# print(f"Stacking Model Accuracy: {accuracy}")